In [61]:
import gpytorch.kernels as kernel
import torch
from src.test_statistics import *
from src.data import *
import random
import numpy as np
from src.test import *

In [62]:
X_ker = kernel.RBFKernel()
Y_ker = kernel.RBFKernel()

In [63]:
n=100
X = torch.randn((n,2))
Y = torch.randn(n)
T = (torch.randn((n))>0).float()

In [64]:
data_train = Data_object(X,Y,T)
data_train_flip = Data_object(X,Y,1-T)

In [65]:
X = torch.randn((n,2))
Y = torch.randn(n)
T = (torch.randn((n))>0).float()

In [66]:
Y1 = torch.randn((1))

In [67]:
data_test = Data_object(X,Y,T)
data_test_flip = Data_object(X,Y,1-T)

In [68]:
W0 = get_W_matrix(X_ker(data_train.X0).evaluate(),1)
W1 = get_W_matrix(Y_ker(data_train.X1).evaluate(),1)

In [69]:
weights = 1/2*torch.ones(n)

In [70]:
K = ker(X_ker)
L = ker(Y_ker)

In [71]:
c = (data_test.T - weights)/(weights*(1-weights))
c

tensor([-2., -2.,  2., -2., -2.,  2., -2., -2., -2.,  2.,  2., -2., -2.,  2.,
         2.,  2.,  2.,  2., -2.,  2., -2., -2.,  2.,  2., -2., -2.,  2., -2.,
        -2.,  2.,  2., -2.,  2., -2.,  2., -2.,  2.,  2., -2., -2., -2.,  2.,
        -2.,  2., -2., -2., -2., -2.,  2.,  2., -2., -2., -2.,  2.,  2.,  2.,
         2., -2., -2., -2.,  2., -2., -2., -2., -2.,  2., -2., -2., -2.,  2.,
        -2., -2., -2., -2.,  2., -2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2., -2.,  2.,  2.,  2.,  2., -2., -2., -2.,  2.,  2., -2., -2.,  2.,
        -2.,  2.])

In [72]:
data_train.Y.shape

torch.Size([100])

In [73]:
L(data_train.Y0,data_train.Y).shape

torch.Size([51, 100])

In [74]:
DATE_test_stat(data_train,data_test,X_ker,Y_ker,weights,W0,W1)

0.022170905023813248

In [75]:
DETT_test_stat(data_train,data_test,X_ker,Y_ker,weights,W1)

0.02217782475054264

In [76]:
from sklearn.linear_model import LogisticRegression

In [77]:
weights_model = LogisticRegression()

In [78]:
weights_model.fit(data_train.X,data_train.T)
weights_train = weights_model.predict_proba(data_train.X)[:,1]
weights_test = weights_model.predict_proba(data_test.X)[:,1]

In [79]:
result_dict = kernel_permutation_test(data_train,data_test,X_ker,Y_ker,weights_model,test_stat="DETT",n_bins=10,n_permutations=100,reg=1)